In [114]:
# build chat input prompt
def build_chat_input(tokenizer, message):
    # chat format:
    # single-turn: <s>Human: Hello!\n\nAssistant: </s>

    prompt = "Human: " + message + "\n\nAssistant:"

    input_tokens = tokenizer.encode(prompt, return_tensors="pt")
    return input_tokens

def build_chat_input_lwm(tokenizer, message, prefill=127*1024):
    # chat format:
    # single-turn: You are a helpful assistant. USER: {} \n ASSISTANT:
    book = tokenizer.encode(message)[:prefill-83]
    prompt = "You are a helpful assistant. USER: Please read a part of the book below, and then give me the summary.\n[start of the book]\n" + tokenizer.decode(book, skip_special_tokens=True) + "\n[end of the book]\n\nNow you have read it. Please summarize it for me. First, tell me the title and the author, and then tell the story in 500 words.\n ASSISTANT:"
    input_tokens = tokenizer.encode(prompt, return_tensors="pt")
    return input_tokens


In [97]:
from datasets import load_dataset

dataset = load_dataset("narrativeqa")

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

In [146]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("LargeWorldModel/LWM-Text-128K")
model = AutoModelForCausalLM.from_pretrained("LargeWorldModel/LWM-Text-128K")

tokenizer_config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/330 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [98]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("JackFram/llama-68m", use_fast=True, legacy=False)

In [109]:
tokenizer.encode("You are a helpful assistant. USER: Please read a part of the book below, and then give me the summary.\n[start of the book]\n\n[end of the book]\n\nNow you have read it. Please summarize it for me. First, tell me the title and the author, and then tell the story in 500 words.\n ASSISTANT:"
, return_tensors="pt").shape

torch.Size([1, 84])

In [110]:
prompt = build_chat_input_lwm(tokenizer, dataset['train'][0]['document']['text'][3:1024*500])
prompt.shape

torch.Size([1, 130048])

In [111]:
tokenizer.decode(prompt[0])

'<s> You are a helpful assistant. USER: Please read a part of the book below, and then give me the summary.\n[start of the book]\nThe Project Gutenberg EBook of Percival Keene, by Frederick Marryat\n\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org\n\n\nTitle: Percival Keene\n\nAuthor: Frederick Marryat\n\nRelease Date: May 22, 2007 [EBook #21572]\n\nLanguage: English\n\n\n*** START OF THIS PROJECT GUTENBERG EBOOK PERCIVAL KEENE ***\n\n\n\n\nProduced by Nick Hodson of London, England\n\n\n\n\nPercival Keene, by Captain Marryat.\n\n________________________________________________________________________\n\nCaptain Frederick Marryat was born July 10 1792, and died August 8 1848.\nHe retired from the British navy in 1828 in order to devote himself to\nwriting.  In the following 20 years h

In [37]:
# prompt = build_chat_input(tokenizer, "Please read a part of the book below, and then give me the summary!\n[start of the book]\n" + dataset['train'][0]['document']['text'][3:1024*356+473] + "\n[end of the book]\n\nNow you have read it! Please summarize it for me!")
prompt = build_chat_input_lwm(tokenizer, dataset['train'][0]['document']['text'][3:1024*470])
prompt.shape

torch.Size([1, 134718])

In [142]:
idx = [0, 50, 300, 800, 950, 1100, 2150, 2450, 2550, 2750, 3350, 3400, 3600, 3900, 4000, 4100, 4200, 4400, 4500, 4550]
tokenized_prompts = []
for i in range(20):
    tokenized_prompt = build_chat_input_lwm(tokenizer, dataset['train'][idx[i]]['document']['text'][3:1024*500])
    if tokenized_prompt.shape[-1] != 127*1024:
        print(i, tokenized_prompt.shape)
        continue
    tokenized_prompts.append(tokenized_prompt)

In [143]:
tokenized_prompts

[tensor([[    1,   887,   526,  ...,  9047, 13566, 29901]]),
 tensor([[    1,   887,   526,  ...,  9047, 13566, 29901]]),
 tensor([[    1,   887,   526,  ...,  9047, 13566, 29901]]),
 tensor([[    1,   887,   526,  ...,  9047, 13566, 29901]]),
 tensor([[    1,   887,   526,  ...,  9047, 13566, 29901]]),
 tensor([[    1,   887,   526,  ...,  9047, 13566, 29901]]),
 tensor([[    1,   887,   526,  ...,  9047, 13566, 29901]]),
 tensor([[    1,   887,   526,  ...,  9047, 13566, 29901]]),
 tensor([[    1,   887,   526,  ...,  9047, 13566, 29901]]),
 tensor([[    1,   887,   526,  ...,  9047, 13566, 29901]]),
 tensor([[    1,   887,   526,  ...,  9047, 13566, 29901]]),
 tensor([[    1,   887,   526,  ...,  9047, 13566, 29901]]),
 tensor([[    1,   887,   526,  ...,  9047, 13566, 29901]]),
 tensor([[    1,   887,   526,  ...,  9047, 13566, 29901]]),
 tensor([[    1,   887,   526,  ...,  9047, 13566, 29901]]),
 tensor([[    1,   887,   526,  ...,  9047, 13566, 29901]]),
 tensor([[    1,   887, 

In [145]:
tokenizer.decode(tokenized_prompts[1][0])

'<s> You are a helpful assistant. USER: Please read a part of the book below, and then give me the summary.\n[start of the book]\nThe Project Gutenberg EBook of Five Weeks in a Balloon, by Jules Verne\n\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org\n\n\nTitle: Five Weeks in a Balloon\n       Journeys and Discoveries in Africa by Three Englishmen\n\nAuthor: Jules Verne\n\nRelease Date: November, 2002 [EBook #3526]\nLast Updated: August 4, 2012\n\nLanguage: English\n\n\n*** START OF THIS PROJECT GUTENBERG EBOOK FIVE WEEKS IN A BALLOON ***\n\n\n\n\nProduced by Judy Boss\n\n\n\n\n\n\nFIVE WEEKS IN A BALLOON\n\nOr,\n\nJourneys And Discoveries In Africa By Three Englishmen.\n\nCompiled In French\n\nBy Jules Verne,\n\nFrom The Original Notes Of Dr. Ferguson.\n\nAnd Done Into English By\n\n

In [140]:
idx = []

for i in range(2000):
    k = i*50
    if dataset['train'][k]['document']['text'][3:1024*500].startswith('The Project Gutenberg'):
        tokenized_prompt = build_chat_input_lwm(tokenizer, dataset['train'][k]['document']['text'][3:1024*500])
        if tokenized_prompt.shape[-1] == 127*1024:
            print(i, tokenized_prompt.shape, dataset['train'][k]['document']['text'][3:1024*500][:60])
            idx.append(k)
            if len(idx) == 20:
                break

0 torch.Size([1, 130048]) The Project Gutenberg EBook of Percival Keene, by Frederick 
1 torch.Size([1, 130048]) The Project Gutenberg EBook of Five Weeks in a Balloon, by J
6 torch.Size([1, 130048]) The Project Gutenberg EBook of The Mysteries of Udolpho, by 
16 torch.Size([1, 130048]) The Project Gutenberg EBook of The Life, Adventures and Pira
19 torch.Size([1, 130048]) The Project Gutenberg EBook of The Helpers, by Francis Lynde
22 torch.Size([1, 130048]) The Project Gutenberg EBook of Dariel, by R. D. Blackmore

T
43 torch.Size([1, 130048]) The Project Gutenberg EBook of The Virginian, by Owen Wister
49 torch.Size([1, 130048]) The Project Gutenberg EBook of The Mysterious Island, by Jul
51 torch.Size([1, 130048]) The Project Gutenberg EBook of Elsie Venner, by Oliver Wende
55 torch.Size([1, 130048]) The Project Gutenberg EBook of The Life and Opinions of Tris
67 torch.Size([1, 130048]) The Project Gutenberg EBook of King Coal, by Upton Sinclair

68 torch.Size([1, 130048]) The Proj

In [ ]:
print(dataset['train'][1450]['document']['text'][3:1024*500])

'The Project Gutenberg eBook of Policeman Bluejay, by L. Frank Baum,\nIllustrated by Maginel Wright Enright\n\n\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org\n\n\n\n\n\nTitle: Policeman Bluejay\n\n\nAuthor: L. Frank Baum\n\n\n\nRelease Date: January 31, 2009  [eBook #27951]\n\nLanguage: English\n\nCharacter set encoding: ISO-646-US (US-ASCII)\n\n\n***START OF THE PROJECT GUTENBERG EBOOK POLICEMAN BLUEJAY***\n\n\nE-text prepared by Michael Gray (Lost_Gamer@comcast.net)\n\n\n\nNote: Project Gutenberg also has an HTML version of this\n      file which includes the original illustrations.\n      See 27951-h.htm or 27951-h.zip:\n      (http://www.gutenberg.net/dirs/2/7/9/5/27951/27951-h/27951-h.htm)\n      or\n      (http://www.gutenberg.net/dirs/2/7/9/5/27951/27951-h.zip)\n\n\n\n\n\nPOL

In [15]:
tokenizer.decode(prompt[0])

'<s> You are a helpful assistant. USER: Please read a part of the book below, and then give me the summary.\n[start of the book]\nThe Project Gutenberg EBook of Percival Keene, by Frederick Marryat\n\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org\n\n\nTitle: Percival Keene\n\nAuthor: Frederick Marryat\n\nRelease Date: May 22, 2007 [EBook #21572]\n\nLanguage: English\n\n\n*** START OF THIS PROJECT GUTENBERG EBOOK PERCIVAL KEENE ***\n\n\n\n\nProduced by Nick Hodson of London, England\n\n\n\n\nPercival Keene, by Captain Marryat.\n\n________________________________________________________________________\n\nCaptain Frederick Marryat was born July 10 1792, and died August 8 1848.\nHe retired from the British navy in 1828 in order to devote himself to\nwriting.  In the following 20 years h